## Storing data in ultra-fast HDF5

In [3]:
!pip install tables

  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.0 MB/s eta 0:00:00a 0:00:01
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)


In [4]:
import warnings

In [5]:
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb

In [6]:
warnings.filterwarnings("ignore")
obb.user.preferences.output_type = "dataframe"

In [7]:
STOCKS_DATA_STORE = "stocks.h5"
FUTURES_DATA_STORE = "futures.h5"

In [8]:
ticker = "SPY"
root = "ES"

Fetches historical price data for the equity "SPY" starting from 2021-01-01 using the "yfinance" provider and stores it in 'spy_equity'

In [9]:
spy_equity = obb.equity.price.historical(
    ticker, start_date="2021-01-01", provider="yfinance"
)

Fetches options chains for the equity "SPY" using the "cboe" provider and stores it in 'spy_chains'

In [13]:
spy_chains = obb.derivatives.options.chains(ticker, provider="yfinance")

Gets the unique expiration dates from 'spy_chains'

In [14]:
spy_expirations = spy_chains.expiration.astype(str).unique().tolist()

Fetches historical price data for a specific option of "SPY" starting from 2021-01-01 using the "yfinance" provider and stores it in 'spy_historic'

In [15]:
spy_historic = obb.equity.price.historical(
    ticker + spy_expirations[-10].replace("-", "")[2:] + "C" + "00400000",
    start_date="2021-01-01",
    provider="yfinance",
)

Save stock data to HDF5 store

In [16]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    store.put("equities/spy/stock_prices", spy_equity)
    store.put("equities/spy/options_prices", spy_historic)
    store.put("equities/spy/chains", spy_chains)

Save futures data to HDF5 store

In [17]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    for i in range(24, 31):
        expiry = f"20{i}-12"
        df = obb.derivatives.futures.historical(
            symbol=[root],
            expiry=expiry,
            start_date="2021-01-01",
        )
        df.rename(columns={"close": expiry}, inplace=True)
        prices = df[expiry]

        store.put(f"futures/{root}/{expiry}", prices)

Load stock data from HDF5 store

In [18]:
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    spy_prices = store["equities/spy/stock_prices"]
    spy_options = store["equities/spy/options_prices"]
    spy_chains = store["equities/spy/chains"]

Load futures data from HDF5 store

In [19]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    es_prices = store[f"futures/{root}/2024-12"]

In [20]:
display(spy_prices)
display(spy_options)
display(es_prices)

,open,high,low,close,volume,dividend
date,,,,,,
2021-01-04,375.309998,375.450012,364.820007,368.790009,110210800,0.0
2021-01-05,368.100006,372.500000,368.049988,371.329987,66426200,0.0
2021-01-06,369.709991,376.980011,369.119995,373.549988,107997700,0.0
2021-01-07,376.100006,379.899994,375.910004,379.100006,68766800,0.0
2021-01-08,380.589996,381.489990,377.100006,381.260010,71677200,0.0
...,...,...,...,...,...,...
2025-01-27,594.809998,599.690002,594.640015,599.369995,70361100,0.0
2025-01-28,600.619995,605.369995,597.250000,604.520020,44433300,0.0
2025-01-29,603.719971,604.130005,599.219971,601.809998,37177400,0.0


,open,high,low,close,volume
date,,,,,
2024-04-04,147.000000,147.000000,147.000000,147.000000,1
2024-05-02,131.429993,131.429993,131.429993,131.429993,4
2024-05-20,156.570007,156.570007,156.320007,156.320007,6
2024-06-03,150.940002,150.940002,150.940002,150.940002,0
2024-06-10,159.199997,159.199997,159.199997,159.199997,1
2024-06-12,168.470001,168.470001,168.470001,168.470001,0
2024-06-17,170.660004,170.660004,170.660004,170.660004,1
2024-06-27,170.169998,170.229996,170.169998,170.229996,3
2024-07-08,178.020004,178.020004,178.020004,178.020004,1


date
2021-01-04    3692.25
2021-01-05    3718.25
2021-01-06    3740.50
2021-01-07    3795.50
2021-01-08    3817.50
               ...   
2025-01-27    6046.75
2025-01-28    6097.00
2025-01-29    6067.50
2025-01-30    6099.25
2025-01-31    6065.50
Name: 2024-12, Length: 1028, dtype: float64

**Jason Strimpel** is the founder of <a href='https://pyquantnews.com/'>PyQuant News</a> and co-founder of <a href='https://www.tradeblotter.io/'>Trade Blotter</a>. His career in algorithmic trading spans 20+ years. He previously traded for a Chicago-based hedge fund, was a risk manager at JPMorgan, and managed production risk technology for an energy derivatives trading firm in London. In Singapore, he served as APAC CIO for an agricultural trading firm and built the data science team for a global metals trading firm. Jason holds degrees in Finance and Economics and a Master's in Quantitative Finance from the Illinois Institute of Technology. His career spans America, Europe, and Asia. He shares his expertise through the <a href='https://pyquantnews.com/subscribe-to-the-pyquant-newsletter/'>PyQuant Newsletter</a>, social media, and has taught over 1,000+ algorithmic trading with Python in his popular course **<a href='https://gettingstartedwithpythonforquantfinance.com/'>Getting Started With Python for Quant Finance</a>**. All code is for educational purposes only. Nothing provided here is financial advise. Use at your own risk.